In [ ]:
!pip install mediapipe
!pip install py7zr
import os
import glob
from pathlib import Path
from csv import writer
import cv2
from cv2 import VideoCapture, waitKey, imshow, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, putText, FONT_HERSHEY_SIMPLEX, resize, rectangle, line, VideoWriter_fourcc, VideoWriter
from numpy import asscalar, ndarray, array, zeros, concatenate
from pathlib import Path
from shutil import rmtree
import mediapipe as mp
import py7zr
import random

In [5]:
with py7zr.SevenZipFile('/content/drive/MyDrive/20_class_video.7z', mode='r') as z:
    z.extractall('/content/ASL dataset')

In [6]:
class MediapipeExtractor:
    
    def extract_landmarks(self, results) -> ndarray:
        result_landmarks = {
            # 'face': {
            #     'landmark': results.face_landmarks,
            #     'shape': (468,)
            # },
            'left hand': {
                'landmark': results.left_hand_landmarks,
                'shape': (21,)
            },
            'right hand': {
                'landmark': results.right_hand_landmarks,
                'shape': (21,)
            },
            'pose': {
                'landmark': results.pose_landmarks,
                'shape': (33,)
            }
        }

        result_all = []
        result_temp = []
        for key, result in result_landmarks.items():
            if result['landmark']:
                result_temp = array([array([l.x, l.y, l.z]) for l in result['landmark'].landmark])
                result_temp = result_temp.flatten()
            else:
                result_temp = zeros((result['shape'][0] * 3, ))

            assert(result_temp.shape == (result['shape'][0] * 3, ))
            result_all.append(result_temp)
            
        shape = 0
        result_final = array([])
        for result in result_all:
            shape += result.shape[0]
            result_final = concatenate((result_final, result), axis=0)

        assert(result_final.shape == (shape,))

        return result_final

In [7]:
class Writer:
    temp_path = ''

    def write_to_csv(self, result: ndarray, out_csv: str):
        out_csv = Path(out_csv)

        with out_csv.open('a+') as csv:
            csv_writer = writer(csv)
            csv_writer.writerow(result)

    def write_as_video(self, path, image):
        if self.temp_path != path: 
            self.temp_path = path
            self.writer = VideoWriter(path, VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))

        self.writer.write(image)


class ImageHandler:
    def draw_results(self, image, results):
        image.flags.writeable = True

        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        return image

In [14]:
me = MediapipeExtractor()
w = Writer()
im = ImageHandler()
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

source_dir = '/content/ASL dataset/video'
out_dir = '/content/test'      #WARNING! MAKE SURE out_dir IS EMPTY OR EVERYTHING WILL BE DELETED



#RUN ON ALL THE FILES IN SOURCE DIRECTORY AND COMPLETELY DELETE OUT THE WHOLE DIRECTORY OF THAT SPECIFIC CLASS IF IT EXISTS

In [11]:
source_directory = Path(source_dir)
classes = list(x.name for x in source_directory.glob('*'))
print(classes)

all_source_ds = []      #source video location list
for category in classes:
    cat_dir = list((source_directory/category).glob("*"))
    all_source_ds.append(cat_dir)
# print(all_source_ds)  #GET READY FOR FLOOOD

counter_class = len(classes)
print(counter_class)        #checking for expected number of classes in source directory
misspartonly = False

['Movie', 'When', 'You', 'ThankYou', 'Class', 'Favourite', 'Where', 'How', 'Your', 'Age', 'What', 'Address', 'Play', 'Work', 'Please', 'School', 'ExcuseMe', 'Name', 'Phone', 'Time']
20


#THIS PART ONLY RUNs ON FILES MISSING IN DESTINATION DIRECTORY, DONT RUN THIS UNLESS REQUIRED. JUST MENTION THE CLASS_NAME


In [ ]:
class_name = 'Class'
src_video_fldr = '/content/ASL dataset/Favourite,ExcuseMe,School,Class_aug/'
csv_folder_with_missing_files = '/content/drive/MyDrive/Favourite,Excuseme,School,Class_csv'

path, dirs, files = next(os.walk(src_video_fldr + class_name))
print('Total Files found : ', len(files)) 
all_source_ds =[]
count =0
for file in files:
    filename = csv_folder_with_missing_files + '/' + class_name + '/' + file
    my_file = Path(filename[:-4]+'.csv')
    if not my_file.exists():
        count +=1
        filename = src_video_fldr + class_name + '/' + file
        all_source_ds.append(filename)
print(all_source_ds[0])
print('Missing files : ' , count)
print(len(all_source_ds))
misspartonly = True
source_directory = Path(src_video_fldr)
classes = [class_name]

Total Files found :  2116
/content/ASL dataset/Favourite,ExcuseMe,School,Class_aug/Class/Class_151.avi
Missing files :  1874
1874


#CSV MAKER on all files


In [17]:
for i in range(len(classes)):
# for i in range(3,4):
    category = classes[i]
    print(i)
    print(category)
    current_ds = all_source_ds[i]   #source video location of i'th class
    print(len(current_ds))
    print(current_ds)
    out_dir_temp = out_dir
    out_dir_temp = out_dir_temp + '/' + classes[i]
    out_directory = Path(out_dir_temp)      #modified out_dir to out_dir -> class name -> csv
    if out_directory.exists(): rmtree(out_directory)        #WARNING! MAKE SURE out_directory IS EMPTY OR EVERYTHING WILL BE DELETED
    out_directory.mkdir()
    progress = 0
    for j in range(len(current_ds)):
        cap = VideoCapture(str(current_ds[j]))
        frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))           
        out_prefix = classes[i] 
        print( "Progress :" + str(progress) + " \\" + str(len(current_ds)) )
        progress+=1
        video_name = os.path.basename(os.path.normpath(str(current_ds[j])))             
        video_name = str(os.path.splitext(video_name)[0])
        print( "New Video loaded" + "\t Category : " + out_prefix + "\t Name : " + video_name + "\n"   )    
        with mp_holistic.Holistic(
            min_detection_confidence = 0.5,
            min_tracking_confidence = 0.5
        ) as holistic:
            while cap.isOpened() and frame_length!=0:
                #print(frame_length)
                image: ndarray
                success, image = cap.read()
                if not success: continue
                image = cvtColor(image, COLOR_BGR2RGB)

                image.flags.writeable = False
                results = holistic.process(image)

                image = cvtColor(image, COLOR_RGB2BGR)
                image = im.draw_results(image, results)

                landmarks = me.extract_landmarks(results)
                print(len(landmarks),landmarks)
                w.write_to_csv(landmarks, out_directory / f'{video_name}.csv')
                frame_length-=1
            break
        break
    break

                # 
                # 

0
Movie
351
[PosixPath('/content/ASL dataset/video/Movie/c (45).avi'), PosixPath('/content/ASL dataset/video/Movie/c (20).avi'), PosixPath('/content/ASL dataset/video/Movie/b (11).avi'), PosixPath('/content/ASL dataset/video/Movie/b (45).avi'), PosixPath('/content/ASL dataset/video/Movie/a (55).avi'), PosixPath('/content/ASL dataset/video/Movie/movie_33.avi'), PosixPath('/content/ASL dataset/video/Movie/movie_40.avi'), PosixPath('/content/ASL dataset/video/Movie/b (96).avi'), PosixPath('/content/ASL dataset/video/Movie/movie_36.avi'), PosixPath('/content/ASL dataset/video/Movie/c (42).avi'), PosixPath('/content/ASL dataset/video/Movie/c (39).avi'), PosixPath('/content/ASL dataset/video/Movie/movie_10.avi'), PosixPath('/content/ASL dataset/video/Movie/Movie_94.avi'), PosixPath('/content/ASL dataset/video/Movie/a (98).avi'), PosixPath('/content/ASL dataset/video/Movie/a (40).avi'), PosixPath('/content/ASL dataset/video/Movie/movie_37.avi'), PosixPath('/content/ASL dataset/video/Movie/c (

In [ ]:
a = 

# CSV FOR MISSING FILES

In [ ]:
if misspartonly == True:
    print(class_name)
    print(len(all_source_ds))   #source video location of missing items from class
    print(all_source_ds)
    out_dir_temp = out_dir + '_missingpart' 
    if not Path(out_dir_temp).exists():
        Path(out_dir_temp).mkdir()
    out_dir_temp = out_dir_temp + '/' + class_name
    out_directory = Path(out_dir_temp)      #modified out_dir to out_dir -> class name -> csv
    if out_directory.exists():      
        out_directory = Path(out_dir_temp + str(random.randint(0,100000)))
    out_directory.mkdir()
    progress = 0
    for j in range(len(all_source_ds)):
        cap = VideoCapture(str(all_source_ds[j]))
        frame_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))           
        out_prefix = class_name
        print( "Progress :" + str(progress) + " \\" + str(len(all_source_ds)) )
        progress+=1
        video_name = os.path.basename(os.path.normpath(str(all_source_ds[j])))             
        video_name = str(os.path.splitext(video_name)[0])
        print( "New Video loaded" + "\t Category : " + out_prefix + "\t Name : " + video_name + "\n"   )    
        with mp_holistic.Holistic(
            min_detection_confidence = 0.5,
            min_tracking_confidence = 0.5
        ) as holistic:
            while cap.isOpened() and frame_length!=0:
                #print(frame_length)
                image: ndarray
                success, image = cap.read()

                if not success: continue
                image = cvtColor(image, COLOR_BGR2RGB)

                image.flags.writeable = False
                results = holistic.process(image)

                image = cvtColor(image, COLOR_RGB2BGR)
                image = im.draw_results(image, results)

                landmarks = me.extract_landmarks(results)

                w.write_to_csv(landmarks, out_directory / f'{video_name}.csv')
                frame_length-=1

Streaming output truncated to the last 5000 lines.
New Video loaded	 Category : Class	 Name : 461458

Progress :208 \1874
New Video loaded	 Category : Class	 Name : Class_207

Progress :209 \1874
New Video loaded	 Category : Class	 Name : Class_225

Progress :210 \1874
New Video loaded	 Category : Class	 Name : 637751

Progress :211 \1874
New Video loaded	 Category : Class	 Name : 625440

Progress :212 \1874
New Video loaded	 Category : Class	 Name : 731037

Progress :213 \1874
New Video loaded	 Category : Class	 Name : 886871

Progress :214 \1874
New Video loaded	 Category : Class	 Name : 701475

Progress :215 \1874
New Video loaded	 Category : Class	 Name : Class_58 (2)

Progress :216 \1874
New Video loaded	 Category : Class	 Name : 625952

Progress :217 \1874
New Video loaded	 Category : Class	 Name : 869271

Progress :218 \1874
New Video loaded	 Category : Class	 Name : 208181

Progress :219 \1874
New Video loaded	 Category : Class	 Name : Class_96

Progress :220 \1874
New Video lo

In [ ]:
!cp /content/drive/MyDrive/Favourite,Excuseme,School,Class_csv_missingpart/Class -r /content/drive/MyDrive/Favourite,Excuseme,School,Class_csv

Delete folder

In [ ]:
!rm -rf /content/drive/MyDrive/Favourite,Excuseme,School,Class_csv_missingpart

In [ ]:
count = 0
from pathlib import Path
source_directory = Path('/content/drive/MyDrive/Favourite,Excuseme,School,Class_csv/School')
files = sorted(list(x.name for x in source_directory.glob('*')))
split_files = []
file = []
for item in files:
    if count%300 == 0 and count!=0:
        split_files.append(file)
        file = []
    file.append(item)
    count+=1
print(count)
print(split_files)

2118
[['100637.csv', '101176.csv', '102081.csv', '102566.csv', '102885.csv', '103104.csv', '103556.csv', '105183.csv', '105469.csv', '106805.csv', '106863.csv', '107376.csv', '107636.csv', '107944.csv', '108387.csv', '108572.csv', '109110.csv', '109728.csv', '110082.csv', '111063.csv', '111195.csv', '111197.csv', '111365.csv', '111701.csv', '112343.csv', '112597.csv', '113188.csv', '113210.csv', '115583.csv', '115600.csv', '115895.csv', '116749.csv', '117266.csv', '117865.csv', '118206.csv', '118605.csv', '118738.csv', '119597.csv', '119696.csv', '119708.csv', '119894.csv', '121080.csv', '121194.csv', '121950.csv', '123131.csv', '123457.csv', '124218.csv', '124840.csv', '125043.csv', '125058.csv', '125720.csv', '125872.csv', '127018.csv', '127677.csv', '127757.csv', '127834.csv', '127859.csv', '128036.csv', '128679.csv', '128753.csv', '128868.csv', '129165.csv', '131039.csv', '131293.csv', '131851.csv', '132178.csv', '133651.csv', '133894.csv', '134724.csv', '134794.csv', '135072.csv',

In [ ]:
import shutil
temp_new_path = Path('/content/When')
for i in range (len(split_files)):
    move_files = split_files[i]
    print(temp_new_path)
    temp_new_path = temp_new_path / str[i]
    temp_new_path.mkdir()
    for j in range(len(move_files)):
        shutil.copy(move_files[i] , temp_new_path)

/content/When


TypeError: ignored

In [ ]:
!py7zr c Favourite,Excuseme,School,Class_csv.7z /content/drive/MyDrive/Favourite,Excuseme,School,Class_csv

In [ ]:
with py7zr.SevenZipFile('Favourite,Excuseme,School,Class_csv.7z', 'w') as archive:
    archive.writeall('/content/drive/MyDrive/Favourite,Excuseme,School,Class_csv')

In [ ]:
!rm -rf /content/drive/MyDrive/What,When,Where,You_csv
!rm -rf /content/drive/MyDrive/What,When,Where,You_csv_missingpart
!rm -rf /content/drive/MyDrive/aug+og
!rm -rf /content/drive/MyDrive/Age,How,ThankYou,Time_csv

In [ ]:
!cp /content/Favourite,Excuseme,School,Class_csv.7z -r /content/drive/MyDrive

In [ ]:
!zip -r /content/file.zip /content/drive/MyDrive/Age,How,ThankYou,Time_csv

In [ ]:
!cp /content/file.zip -r /content/drive/MyDrive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pandas import DataFrame, read_csv
from pathlib import Path
from random import shuffle
import random
from os.path import split
db = Path('/content/drive/MyDrive/What,When,Where,You_csv')
path_to_source_folder = Path('/content/drive/MyDrive/What,When,Where,You_csv')
selected_db = db
source_directory = path_to_source_folder
y = []
x = []
f = []
e = []
all_source_ds =[]
x_valid =[]
y_valid = []
f_valid =[]
failed_final_check =[]
batch_size = 100
ACCURACY_THRESHOLD = 0.95
split_ratio = 0.2     
classes = sorted(list(x.name for x in source_directory.glob('*')))
if any('.ipynb_checkpoints' in s for s in classes):
    classes.remove('.ipynb_checkpoints')
print(classes)
for category in classes:
    if category != 'When':
        cat_dir = list((source_directory/category).glob("*"))
        all_source_ds.append(cat_dir)
print(all_source_ds)
all_source_ds_flat = [item for sublist in all_source_ds for item in sublist]
shuffle(all_source_ds_flat)
print(len(all_source_ds_flat))
valid_csv_list = all_source_ds_flat[:int(len(all_source_ds_flat)*split_ratio)]
train_csv_list = all_source_ds_flat[int(len(all_source_ds_flat)*split_ratio):]
print(all_source_ds_flat)
print(len(all_source_ds_flat))
for csv in valid_csv_list:
    d = read_csv(str(csv), header=None ).values
    if d.shape == (50,1629):
            random_number = random.randint(1,30)
            x_valid.append(d[random_number:random_number+20])
            label = split(split(csv)[0])[1]
            print(csv,"  ", label)
            y_valid.append(label)
    else:
            f_valid.append(d)
            print(d.shape)

['What', 'When', 'Where', 'You']
[[PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/681121.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/What_298.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/883210.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/992244.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/220187.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/970952.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/393436.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/699269.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/409469.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/966233.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/838112.csv'), PosixPath('/content/drive/MyDrive/What,When,Where,You_csv/What/What_239.csv'), PosixPath('/content/drive/MyDrive/Wha

ParserError: ignored

In [ ]:
from shutil import rmtree
rmtree(Path('/content/fruits'))
path = Path('/content/fruits')
path.mkdir()
path = Path('/content/fruits/apple_1')
path.mkdir()

In [ ]:
from pathlib import Path
for i in range (0,50):
    path = Path('/content/fruiits/apple' + str(i))
    path.mkdir()

FileNotFoundError: ignored

In [ ]:
def uniquify(path,class_name):
    filename, extension = os.path.splitext(path)
    print('filename', filename, "extension ", extension)
    counter = 1
    while os.path.exists(path):
        path = filename + '/' + class_name + '_' + str(counter) + extension
        print(path)
        counter += 1
    print(counter)
    return counter-1

In [ ]:
import os
for i in range (0,10):
    class_name = 'apple'
    path = Path('/content/fruits/')
    cur_video_count = uniquify(path,class_name)
    path = path / str('apple_' + str(cur_video_count))
    Path(path).mkdir()

In [ ]:
def uniquify(path):
    counter = 0
    label = split(split(path)[1])[1]
    temp_path = path + '/' + label + '_' + str(counter) + '.csv'
    print(temp_path)
    while os.path.exists(temp_path):
        counter += 1
        temp_path = path + '/' + label + '_' + str(counter) + '.csv'
        print(temp_path)
    return counter

In [ ]:
cur_video_count = uniquify("/content/What")
print(cur_video_count)

/content/What/What_0.csv
/content/What/What_1.csv
/content/What/What_2.csv
/content/What/What_3.csv
3


In [ ]:
from os.path import split
label = split(split("D:/ASL_DATASET/recording/csv/What.csv")[1])[1]
print(label)

What


In [ ]:
print(label)

csv
